In [ ]:
# 백테스팅 결과 시각화
if 'cumulative_returns' in locals():
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 누적수익률 차트
    axes[0, 0].plot(cumulative_returns.index, (cumulative_returns - 1) * 100)
    axes[0, 0].set_title('누적수익률 비교 (%)', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('누적수익률 (%)')
    axes[0, 0].legend(cumulative_returns.columns, loc='upper left', fontsize=8)
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 경기국면별 선택 스타일 분포
    if 'style_selections' in locals():
        style_dist = pd.Series(style_selections).value_counts()
        axes[0, 1].pie(style_dist.values, labels=style_dist.index, autopct='%1.1f%%')
        axes[0, 1].set_title('RSI 로테이션 전략 스타일 선택 분포', fontsize=14, fontweight='bold')
    
    # 3. 연도별 수익률 비교
    annual_returns = strategies_comparison.groupby(strategies_comparison.index.year).apply(
        lambda x: (1 + x).prod() - 1
    ) * 100
    
    x_pos = np.arange(len(annual_returns.index))
    width = 0.15
    
    for i, strategy in enumerate(annual_returns.columns):
        axes[1, 0].bar(x_pos + i * width, annual_returns[strategy], width, 
                      label=strategy, alpha=0.8)
    
    axes[1, 0].set_xlabel('연도')
    axes[1, 0].set_ylabel('연간 수익률 (%)')
    axes[1, 0].set_title('연도별 수익률 비교', fontsize=14, fontweight='bold')
    axes[1, 0].set_xticks(x_pos + width * (len(annual_returns.columns) - 1) / 2)
    axes[1, 0].set_xticklabels(annual_returns.index)
    axes[1, 0].legend(fontsize=8)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 위험-수익률 산점도
    risk_return_data = performance_metrics[['연율화수익률(%)', '변동성(%)']].copy()
    
    axes[1, 1].scatter(risk_return_data['변동성(%)'], risk_return_data['연율화수익률(%)'], 
                      s=100, alpha=0.7)
    
    for i, strategy in enumerate(risk_return_data.index):
        axes[1, 1].annotate(strategy, 
                           (risk_return_data.iloc[i]['변동성(%)'], 
                            risk_return_data.iloc[i]['연율화수익률(%)']),
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    axes[1, 1].set_xlabel('변동성 (%)')
    axes[1, 1].set_ylabel('연율화 수익률 (%)')
    axes[1, 1].set_title('위험-수익률 프로파일', fontsize=14, fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # RSI 신호와 선택된 스타일 타임라인
    if 'combined_data' in locals() and 'style_selections' in locals():
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
        
        # RSI와 경기국면
        ax1.plot(combined_data.index, combined_data['RSI'], color='blue', linewidth=2)
        ax1.axhline(y=50, color='red', linestyle='--', alpha=0.7, label='RSI 50선')
        ax1.set_ylabel('RSI')
        ax1.set_title('RSI 지표와 경기국면 분류', fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 경기국면별 색상 배경
        season_colors = {
            '🌸 봄(상승기)': 'lightgreen',
            '☀️ 여름(호황기)': 'yellow', 
            '🍂 가을(하락기)': 'orange',
            '❄️ 겨울(침체기)': 'lightblue'
        }
        
        for i in range(len(combined_data) - 1):
            season = combined_data['경기국면'].iloc[i]
            if pd.notna(season) and season in season_colors:
                ax1.axvspan(combined_data.index[i], combined_data.index[i+1], 
                           alpha=0.2, color=season_colors[season])
        
        # 선택된 스타일 타임라인
        style_codes = {style: i for i, style in enumerate(set(style_selections))}
        style_timeline = [style_codes.get(style, 0) for style in style_selections]
        
        ax2.plot(returns_data.index, style_timeline, 'o-', linewidth=2, markersize=4)
        ax2.set_ylabel('선택된 스타일')
        ax2.set_xlabel('날짜')
        ax2.set_title('RSI 로테이션 전략 - 선택된 스타일 타임라인', fontsize=14, fontweight='bold')
        ax2.set_yticks(list(style_codes.values()))
        ax2.set_yticklabels(list(style_codes.keys()), rotation=45)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
else:
    print("누적수익률 데이터가 없습니다. 이전 셀을 먼저 실행해주세요.")

In [ ]:
# 백테스팅 성과지표 계산
if 'strategies_comparison' in locals():
    
    def calculate_performance_metrics(returns_series):
        \"\"\"성과지표 계산 함수\"\"\"
        total_return = (1 + returns_series).prod() - 1
        annualized_return = (1 + total_return) ** (12 / len(returns_series)) - 1
        volatility = returns_series.std() * np.sqrt(12)
        sharpe_ratio = annualized_return / volatility if volatility > 0 else 0
        
        # 최대 낙폭 계산
        cumulative = (1 + returns_series).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative / running_max) - 1
        max_drawdown = drawdown.min()
        
        # 승률 계산
        win_rate = (returns_series > 0).mean()
        
        return {
            '총수익률(%)': total_return * 100,
            '연율화수익률(%)': annualized_return * 100,
            '변동성(%)': volatility * 100,
            '샤프비율': sharpe_ratio,
            '최대낙폭(%)': max_drawdown * 100,
            '승률(%)': win_rate * 100,
            '관측수': len(returns_series)
        }
    
    # 전략별 성과지표 계산
    performance_metrics = pd.DataFrame()
    
    for strategy in strategies_comparison.columns:
        returns = strategies_comparison[strategy].dropna()
        metrics = calculate_performance_metrics(returns)
        performance_metrics[strategy] = metrics
    
    performance_metrics = performance_metrics.T
    
    print("=== 백테스팅 성과지표 비교 ===")
    print(performance_metrics.round(2).to_string())
    
    # 누적수익률 계산
    cumulative_returns = (1 + strategies_comparison.fillna(0)).cumprod()
    
    print(f"\n=== 기간별 누적수익률 ===")
    print(f"최종 누적수익률:")
    for strategy in cumulative_returns.columns:
        final_return = (cumulative_returns[strategy].iloc[-1] - 1) * 100
        print(f"{strategy}: {final_return:.2f}%")
        
else:
    print("전략 비교 데이터가 없습니다. 이전 셀을 먼저 실행해주세요.")

In [ ]:
# RSI 기반 스타일 로테이션 전략 구현
if 'returns_with_season' in locals():
    
    # 계절별 최고 성과 스타일 식별
    best_styles_by_season = {}
    
    for season in seasonal_performance.keys():
        # 샤프비율 기준으로 최고 성과 스타일 선택
        season_sharpe = seasonal_performance[season]['샤프비율']
        best_style = season_sharpe.idxmax()
        best_styles_by_season[season] = best_style
        print(f"{season}: {best_style} (샤프비율: {season_sharpe[best_style]:.3f})")
    
    # 로테이션 전략 백테스팅
    def backtest_rotation_strategy(returns_data, season_data, style_mapping):
        \"\"\"RSI 기반 스타일 로테이션 전략 백테스팅\"\"\"
        portfolio_returns = []
        selected_styles = []
        
        for date, season in season_data.items():
            if pd.notna(season) and season in style_mapping:
                selected_style = style_mapping[season]
                if selected_style in returns_data.columns:
                    monthly_return = returns_data.loc[date, selected_style]
                    portfolio_returns.append(monthly_return)
                    selected_styles.append(selected_style)
                else:
                    portfolio_returns.append(0)
                    selected_styles.append('현금')
            else:
                portfolio_returns.append(0)
                selected_styles.append('현금')
        
        return pd.Series(portfolio_returns, index=returns_data.index), selected_styles
    
    # 로테이션 전략 실행
    rotation_returns, style_selections = backtest_rotation_strategy(
        returns_data, 
        combined_data['경기국면'].iloc[1:], 
        best_styles_by_season
    )
    
    # 벤치마크 전략들
    benchmark_strategies = {}
    
    # 1. 균등가중 포트폴리오
    equal_weight_returns = returns_data[style_columns].mean(axis=1)
    benchmark_strategies['균등가중'] = equal_weight_returns
    
    # 2. 개별 스타일 Buy & Hold
    for style in style_columns:
        benchmark_strategies[f'{style} B&H'] = returns_data[style]
    
    # 성과 비교
    strategies_comparison = pd.DataFrame()
    strategies_comparison['RSI 로테이션'] = rotation_returns
    
    for name, returns in benchmark_strategies.items():
        strategies_comparison[name] = returns
    
    print(f"\n=== RSI 기반 스타일 로테이션 전략 ===")
    print(f"백테스팅 기간: {returns_data.index[0].strftime('%Y-%m')} ~ {returns_data.index[-1].strftime('%Y-%m')}")
    print(f"총 {len(rotation_returns)}개월")
    
else:
    print("수익률 데이터가 없습니다. 이전 셀을 먼저 실행해주세요.")

In [ ]:
# 계절별 스타일지수 성과 분석
if 'combined_data' in locals():
    # 스타일지수 컬럼 식별
    style_columns = [col for col in combined_data.columns if col not in ['경기국면', 'RSI', 'RSI_direction']]
    
    print(f"분석할 스타일지수: {style_columns}")
    
    # 월별 수익률 계산
    returns_data = combined_data[style_columns].pct_change().dropna()
    
    # RSI 분류와 수익률 데이터 병합
    returns_with_season = pd.concat([returns_data, combined_data[['경기국면']].iloc[1:]], axis=1)
    
    # 계절별 평균 수익률 계산
    seasonal_performance = {}
    
    for season in returns_with_season['경기국면'].unique():
        if pd.notna(season):
            season_data = returns_with_season[returns_with_season['경기국면'] == season]
            seasonal_performance[season] = {
                '평균수익률': season_data[style_columns].mean(),
                '표준편차': season_data[style_columns].std(),
                '샤프비율': season_data[style_columns].mean() / season_data[style_columns].std(),
                '관측수': len(season_data)
            }
    
    # 계절별 성과 요약 테이블 생성
    performance_summary = pd.DataFrame()
    
    for season, metrics in seasonal_performance.items():
        temp_df = pd.DataFrame({
            '계절': season,
            '스타일': style_columns,
            '평균수익률(%)': (metrics['평균수익률'] * 100).round(2),
            '변동성(%)': (metrics['표준편차'] * 100).round(2),
            '샤프비율': metrics['샤프비율'].round(3),
            '관측수': metrics['관측수']
        })
        performance_summary = pd.concat([performance_summary, temp_df], ignore_index=True)
    
    print("\n=== 계절별 스타일지수 성과 요약 ===")
    print(performance_summary.to_string(index=False))
    
else:
    print("통합 데이터가 없습니다. 이전 셀을 먼저 실행해주세요.")

In [ ]:
# RSI 분류와 스타일지수 데이터 통합
# 기존 RSI 분류 결과와 스타일지수 데이터를 날짜 기준으로 병합

# RSI 분류 결과 준비 (기존에 계산된 것 사용)
if 'rsi_classification' in locals():
    # RSI 분류 데이터를 월별로 리샘플링
    rsi_monthly = rsi_classification.resample('M').last()
    
    # 공통 기간 찾기
    common_start = max(style_monthly.index[0], rsi_monthly.index[0])
    common_end = min(style_monthly.index[-1], rsi_monthly.index[-1])
    
    # 공통 기간으로 데이터 자르기
    style_aligned = style_monthly.loc[common_start:common_end]
    rsi_aligned = rsi_monthly.loc[common_start:common_end]
    
    # 데이터 통합
    combined_data = pd.concat([style_aligned, rsi_aligned[['경기국면', 'RSI', 'RSI_direction']]], axis=1)
    
    print(f"통합 데이터 기간: {common_start.strftime('%Y-%m')} ~ {common_end.strftime('%Y-%m')}")
    print(f"총 {len(combined_data)}개월 데이터")
    print(f"\n경기국면별 분포:")
    print(combined_data['경기국면'].value_counts())
    
else:
    print("RSI 분류 결과가 없습니다. 먼저 RSI 계산 셀을 실행해주세요.")

In [ ]:
# RSI 기반 스타일지수 백테스팅 시스템
print("=== RSI 기반 S&P 스타일지수 백테스팅 시스템 ===")

# 스타일지수 데이터와 RSI 분류 통합
style_data = pd.read_excel(r"C:\Users\wnghk\Desktop\ACADEMY\2025-1.5\계량경제\스타일지수 시뮬레이션.xlsx", 
                          sheet_name=0, index_col=0, parse_dates=True)

print(f"스타일지수 데이터 기간: {style_data.index[0].strftime('%Y-%m')} ~ {style_data.index[-1].strftime('%Y-%m')}")
print(f"스타일지수 종류: {list(style_data.columns)}")

# 월별 데이터로 맞추기 (월말 기준)
style_monthly = style_data.resample('M').last()

print(f"\n월별 스타일지수 데이터 형태: {style_monthly.shape}")
print(f"기간: {style_monthly.index[0].strftime('%Y-%m')} ~ {style_monthly.index[-1].strftime('%Y-%m')}")

### 4개국면 구분을 위한 은닉마르코프 모델
1. 데이터 기간
- 1998.08 ~ 2025.04 (월 기준) 

2. 사용 변수

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")

## 시각화
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')


In [ ]:
## Load Data
file_path = 'C:/Users/wnghk/Desktop/ACADEMY/2025-1.5/계량경제/'
df = pd.read_excel(file_path+'Monthly_data_v1.0.xlsx')
df.head()

In [3]:
df_cols = [col for col in df.columns if col != 'date']
df_cols

['S&P500',
 'S&P500_rate',
 'UST10Y',
 'VIX_SPX',
 'TermSpread',
 'CreditSpread',
 'DXY',
 'MarginDebt',
 'MOVE',
 'UnempRate',
 'ExpInfl10Y',
 'CS_HPI',
 'GoldSpot']

In [4]:
# 상관계수(피어슨) 계산
corr_matrix = df[df_cols].corr()  # 기본은 Pearson 상관계수
print(corr_matrix)

                S&P500  S&P500_rate    UST10Y   VIX_SPX  TermSpread  \
S&P500        1.000000     0.090821  0.081847  0.029290   -0.013998   
S&P500_rate   0.090821     1.000000  0.156717 -0.693923   -0.054082   
UST10Y        0.081847     0.156717  1.000000 -0.116937    0.391337   
VIX_SPX       0.029290    -0.693923 -0.116937  1.000000    0.056985   
TermSpread   -0.013998    -0.054082  0.391337  0.056985    1.000000   
CreditSpread -0.022406    -0.493282 -0.502624  0.383513    0.012671   
DXY           0.026270    -0.325845  0.225975  0.217854   -0.054471   
MarginDebt    0.025165     0.534476  0.237150 -0.274213    0.047192   
MOVE          0.018282    -0.357719  0.110161  0.486872    0.218596   
UnempRate    -0.037516     0.075208 -0.033224 -0.083017    0.014937   
ExpInfl10Y   -0.007911     0.288258  0.476945 -0.174270    0.233643   
CS_HPI        0.156993     0.070297  0.087942 -0.029497   -0.097547   
GoldSpot      0.033105     0.050799 -0.273214 -0.027844   -0.063986   

     

-----
#### 1. 기초데이터 분석
##### 1) 데이터 전처리

In [5]:
## 데이터 확인
print("1. row, col : ", df.shape)    # (321, 13)

## 결측치/이상치 확인 및 처리
df_cols = [col for col in df.columns]
print("2. 변수별 결측치\n", df[df_cols].isnull().sum())

# 날짜 컬럼 변환
df['date'] = pd.to_datetime(df['date'], format='%Y%m')

1. row, col :  (321, 14)
2. 변수별 결측치
 date            0
S&P500          0
S&P500_rate     0
UST10Y          0
VIX_SPX         0
TermSpread      0
CreditSpread    0
DXY             0
MarginDebt      0
MOVE            0
UnempRate       0
ExpInfl10Y      0
CS_HPI          0
GoldSpot        0
dtype: int64


In [ ]:
## 더미화

## 기대인플레이션 단순 2% 기준
# df['ExpInfl10Y_dummy'] = (df['ExpInfl10Y'] >= 0.2).astype(int)

## 소비자
# 조건: (직전월 < 5%) & (당월 >= 5%)인 달만 1, 아니면 0
# 1. 직전월 시리즈 생성 (shift)
prev = df['UnempRate'].shift(1)

# 2. 조건 결합
df['UnempRate_5over_entry'] = ((prev < 5) & (df['UnempRate'] >= 5)).astype(int)

## 2% 이상 인플레이션
# 이하 인플레이션 x

In [ ]:
# 6개 변수 지정 (VIX_SPX, UST10Y, DXY, MOVE, GoldSpot, MarginDebt)
variables = ['VIX_SPX', 'UST10Y', 'DXY', 'MOVE', 'GoldSpot', 'MarginDebt']
print(f"지정된 6개 변수: {variables}")

# 별도 변수값으로 사용
sp500 = df['S&P500'].values
dates = df['date'].values

In [17]:
## 기초통계
for col in variables:
    print(f"{col} - 평균: {df[col].mean():.4f}, 표준편차: {df[col].std():.4f}, min: {df[col].min()}, max: {df[col].max()}")

# 상관계수(피어슨) 계산
corr_matrix = df[variables].corr()  # 기본은 Pearson 상관계수
print(corr_matrix)

S&P500_rate - 평균: 0.0060, 표준편차: 0.0448, min: -0.16942453444905514, max: 0.12684410293315374
UST10Y - 평균: -0.0041, 표준편차: 0.2638, min: -1.0396, max: 0.8729999999999998
VIX_SPX - 평균: 0.0234, 표준편차: 0.2324, min: -0.4589689637033141, max: 1.3457095709570956
CreditSpread - 평균: 0.0539, 표준편차: 23.1643, min: -82.07, max: 156.04000000000002
MarginDebt - 평균: 0.0062, 표준편차: 0.0449, min: -0.197188679518142, max: 0.12377804889348765
MOVE - 평균: 0.0157, 표준편차: 0.1759, min: -0.36103493501848094, max: 1.0242718446601944
UnempRate - 평균: -0.0009, 표준편차: 0.6352, min: -2.1999999999999993, max: 10.4
CS_HPI - 평균: 0.0041, 표준편차: 0.0079, min: -0.022553982187479837, max: 0.0271653954654687
GoldSpot - 평균: 0.0087, 표준편차: 0.0467, min: -0.16889603306734025, max: 0.16845275344180233
ExpInfl10Y_dummy - 평균: 0.0966, 표준편차: 0.2958, min: 0, max: 1
                  S&P500_rate    UST10Y   VIX_SPX  CreditSpread  MarginDebt  \
S&P500_rate          1.000000  0.156717 -0.693923     -0.493282    0.534476   
UST10Y               0.1567

In [ ]:
## Feature selection (6개 변수 기준)

# 먼저 데이터 컬럼 확인
print("데이터 컬럼 목록:")
print(df.columns.tolist())

# S&P500_rate가 있는지 확인하고 대안 사용
if 'S&P500_rate' in df.columns:
    y_col = 'S&P500_rate'
elif 'S&P500' in df.columns:
    # S&P500 수익률을 직접 계산
    df['S&P500_rate_calc'] = df['S&P500'].pct_change()
    y_col = 'S&P500_rate_calc'
else:
    print("S&P500 관련 컬럼을 찾을 수 없습니다.")
    y_col = None

if y_col:
    from sklearn.linear_model import LassoCV
    
    # 6개 변수와 S&P500 수익률 간의 관계 분석
    X = df[variables]
    y = df[y_col]
    
    # 결측치 처리
    common_idx = X.dropna().index.intersection(y.dropna().index)
    X_clean = X.loc[common_idx]
    y_clean = y.loc[common_idx]
    
    if len(X_clean) > 0:
        lasso = LassoCV(cv=5).fit(X_clean, y_clean)
        importance = pd.Series(lasso.coef_, index=variables)
        print(f"\n6개 변수의 {y_col} 예측 중요도:")
        print(importance.sort_values(ascending=False).round(6))
    else:
        print("분석할 데이터가 없습니다.")
else:
    print("Feature selection을 건너뜁니다.")

In [ ]:
# ## 필요시 ----------------------------------------------------------

# # 결측치 처리 → 결측치가 있으면 보간/제거. 일반적으로 선형보간 사용
# df[variables] = df[variables].interpolate(method='linear')
# df = df.dropna(subset=variables).reset_index(drop=True)


# # 로그 변환 : 성장률 분석에 적합, 분포 안정화

# rate_cols = ['GoldSpot']

# for col in df.columns:
#     if col in rate_cols :
#         # 등락률(변화율) 계산
#         df[f'rate_{col}'] = df[col].pct_change() * 100     # 퍼센트 단위
    
# # 로그 변환
# df[f'log_{col}'] = np.log(df[col])

# df = df.dropna().reset_index(drop=True)  # 시작값은 null로 제거
# df

# # 차분 : 비정상성 제거, 수준에서 성장률로 변환
# # diff_data = df.diff().dropna()
# # diff_data

,date,UST10Y,VIX_SPX,TermSpread,CreditSpread,DXY,MOVE,UnempRate,ExpInfl10Y,CS_HPI,GoldSpot,log_GoldSpot,rate_GoldSpot
0,2025-03-01,4.211,22.28,31.799,179.09,104.210,101.35,4.2,2.3703,327.60,3123.57,8.046732,-5.021422
1,2025-02-01,4.212,19.63,21.523,160.18,107.614,104.46,4.1,2.3677,325.09,2857.83,7.957818,-8.507573
2,2025-01-01,4.543,16.43,33.749,143.13,108.370,91.76,4.0,2.4302,323.65,2798.41,7.936807,-2.079200
3,2024-12-01,4.575,17.35,32.529,141.10,108.487,98.80,4.1,2.3388,323.36,2624.50,7.872646,-6.214600
4,2024-11-01,4.178,13.51,1.347,156.15,105.737,95.22,4.2,2.2679,323.77,2643.15,7.879727,0.710612
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,1998-12-01,4.650,24.42,9.400,258.20,94.170,121.13,4.4,0.8019,92.45,288.25,5.663828,0.733881
316,1998-11-01,4.710,26.01,27.500,247.60,96.200,87.22,4.4,0.9815,92.20,293.20,5.680855,1.717259
317,1998-10-01,4.595,28.05,32.800,268.50,93.680,121.06,4.5,0.9827,92.00,292.55,5.678636,-0.221692
318,1998-09-01,4.395,40.95,27.050,259.00,96.170,124.22,4.6,0.8509,91.76,296.95,5.693564,1.504016


-------
#### 2. 모델링 : 은닉 마르코프 체인
- Multinomial HMM : input이 discrete
- GaussianHMM : input이 continuous
- GMMHMM : 여러개의 gaussian 모델로 확장된 모델

-----
##### 2) GaussianHMM

In [7]:
## 데이터 복제
df2 = df.copy()
variables

['S&P500_rate',
 'UST10Y',
 'VIX_SPX',
 'CreditSpread',
 'MarginDebt',
 'MOVE',
 'UnempRate',
 'ExpInfl10Y',
 'CS_HPI',
 'GoldSpot']

In [ ]:
# 6개 변수에 맞게 전처리 방식 수정
# %p형 변수 리스트 (포인트 단위 변수)
pp_vars = ['UST10Y']  # 금리 변수만

# % 변화율형 변수 리스트 (이미 변화율이거나 지수형)
pct_vars = ['VIX_SPX', 'DXY', 'MOVE', 'GoldSpot', 'MarginDebt']

# %p형 변수를 변화율(%)로 변환: (이번달-전달)/전달*100
for col in pp_vars:
    df2[f'{col}_chg_pct'] = df2[col].pct_change() * 100
    
all_vars = [f'{col}_chg_pct' for col in pp_vars] + pct_vars
df2[all_vars] = df2[all_vars].replace([np.inf, -np.inf], np.nan)

# NaN 또는 inf 포함된 행 전체를 drop
df2 = df2.dropna(subset=all_vars).reset_index(drop=True)

print(f"최종 사용 변수: {all_vars}")
print(f"데이터 크기: {df2.shape}")

In [ ]:
## scaling (6개 변수)
scaler = StandardScaler()

# 6개 변수로 스케일링
df_scaled = scaler.fit_transform(df2[all_vars])

print(f"스케일링 완료: {df_scaled.shape}")
print(f"변수 순서: {all_vars}")

In [ ]:
## HMM 모델링 및 상태 분류 (6개 변수)
from hmmlearn.hmm import GaussianHMM

# 상태 개수 지정 (4개 국면)
n_states = 4

# HMM 모델 학습
model = GaussianHMM(n_components=n_states, covariance_type="full", n_iter=1000, random_state=42)
model.fit(df_scaled)

# 상태 추정
hidden_states = model.predict(df_scaled)
df2['state'] = hidden_states

# 2. 상태별(국면별) 변수 특성(평균, 표준편차 등) 확인
print("6개 변수 HMM 분석 결과:")
for i in range(n_states):
    print(f"\nState {i}: 빈도 = {(hidden_states==i).sum()}")
    for j, col in enumerate(variables):
        mean = df2.loc[hidden_states==i, col].mean()
        std = df2.loc[hidden_states==i, col].std()
        print(f"  {col}: 평균={mean:.4f}, 표준편차={std:.4f}")

# 전이확률 행렬 및 상태별 통계
print("\n전이확률행렬:")
print(pd.DataFrame(model.transmat_.round(3)))
print("\n상태별 평균 (표준화된 값):")
print(pd.DataFrame(model.means_.round(3), columns=all_vars))

In [ ]:
## 시각화 (개선된 3상태 모델과 원래 4상태 모델 비교)

# 두 모델 모두 시각화
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# 유효한 인덱스와 데이터 준비
valid_idx = df2[variables].dropna().index
dates_valid = df2.loc[valid_idx, 'date']
sp500_valid = df2.loc[valid_idx, 'S&P500']

# 1. 원래 4상태 모델 시각화
ax1 = axes[0, 0]
ax1.plot(dates_valid, sp500_valid, color='gray', alpha=0.7, label='S&P500 Index', linewidth=1)

state_colors_4 = ['blue', 'orange', 'green', 'red']
for state in range(n_states):
    idx = hidden_states == state
    ax1.scatter(dates_valid[idx], sp500_valid.values[idx], 
               s=15, color=state_colors_4[state], label=f'State {state}', alpha=0.8)

ax1.set_title("원래 4상태 모델 결과")
ax1.set_xlabel("Date")
ax1.set_ylabel("S&P500 Index")
ax1.legend()
ax1.xaxis.set_major_locator(mdates.YearLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

# 2. 개선된 3상태 모델 시각화
ax2 = axes[0, 1]
ax2.plot(dates_valid, sp500_valid, color='gray', alpha=0.7, label='S&P500 Index', linewidth=1)

state_colors_3 = ['blue', 'orange', 'green']

# 3상태 데이터 확인 및 시각화
print("3상태 모델 데이터 확인:")
if 'state_improved' in df2.columns:
    print("state_improved 컬럼 있음")
    states_3 = df2['state_improved'].values
elif 'states_improved' in locals():
    print("states_improved 변수 사용")
    states_3 = states_improved
    df2['state_improved'] = states_improved
else:
    print("3상태 데이터 없음 - 다시 생성")
    # 3상태 모델 다시 생성
    from hmmlearn.hmm import GaussianHMM
    model_3 = GaussianHMM(n_components=3, covariance_type="diag", n_iter=1000, random_state=42)
    model_3.fit(df_scaled)
    states_3 = model_3.predict(df_scaled)
    df2['state_improved'] = states_3

print(f"3상태 분포: {pd.Series(states_3).value_counts().sort_index()}")

# 3상태 모델 시각화
for state in range(3):
    idx = states_3 == state
    if np.any(idx):
        ax2.scatter(dates_valid[idx], sp500_valid.values[idx], 
                   s=15, color=state_colors_3[state], label=f'State {state}', alpha=0.8)

ax2.set_title("개선된 3상태 모델 결과")
ax2.set_xlabel("Date")
ax2.set_ylabel("S&P500 Index")
ax2.legend()
ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

# 3. 상태별 분포 비교 (4상태)
ax3 = axes[1, 0]
state_counts_4 = pd.Series(hidden_states).value_counts().sort_index()
ax3.pie(state_counts_4.values, labels=[f'State {i}\n({count}개월)' for i, count in state_counts_4.items()], 
        autopct='%1.1f%%', colors=state_colors_4[:len(state_counts_4)])
ax3.set_title('4상태 모델 분포\n(불균형 문제)')

# 4. 상태별 분포 비교 (3상태)
ax4 = axes[1, 1]
state_counts_3 = pd.Series(states_3).value_counts().sort_index()
ax4.pie(state_counts_3.values, labels=[f'State {i}\n({count}개월)' for i, count in state_counts_3.items()], 
        autopct='%1.1f%%', colors=state_colors_3[:len(state_counts_3)])
ax4.set_title('3상태 모델 분포\n(균형 개선)')

plt.tight_layout()
plt.show()

# 상세 비교 분석
print("\n=== 4상태 vs 3상태 모델 비교 ===")

print("\n1. 상태별 분포 비교:")
print("4상태 모델:")
for state, count in state_counts_4.items():
    percentage = count / len(hidden_states) * 100
    status = "❌ 불균형" if percentage < 5 else "✅ 정상"
    print(f"  State {state}: {count}개월 ({percentage:.1f}%) {status}")

print("\n3상태 모델:")
for state, count in state_counts_3.items():
    percentage = count / len(states_3) * 100
    status = "✅ 정상" if percentage >= 5 else "❌ 불균형"
    print(f"  State {state}: {count}개월 ({percentage:.1f}%) {status}")

# 균형도 점수 비교
balance_4 = state_counts_4.std()
balance_3 = state_counts_3.std()
print(f"\n2. 균형도 점수 (낮을수록 좋음):")
print(f"  4상태 모델: {balance_4:.1f}")
print(f"  3상태 모델: {balance_3:.1f}")
print(f"  개선도: {((balance_4 - balance_3) / balance_4 * 100):.1f}% 향상")

# 경제적 해석성 비교
print(f"\n3. 경제적 해석성:")
print("4상태 모델:")
print("  ✅ 4계절 국면 구분 가능")
print("  ❌ 2개 상태가 극소수로 의미 없음")
print("  ❌ 과적합 우려")

print("\n3상태 모델:")
print("  ✅ 위기-정상-회복 3단계 명확")
print("  ✅ 모든 상태가 의미있는 크기")
print("  ✅ 안정적 상태 전이")

print(f"\n4. 권장사항:")
print("📊 3상태 모델 사용 권장")
print("🎯 더 안정적이고 해석 가능한 시장 국면 분류")
print("💡 향후 투자전략 도출에 더 적합")

In [ ]:
## 스타일지수 시뮬레이션 데이터 확인

print("=== 스타일지수 시뮬레이션 데이터 로드 ===")

# 스타일지수 시뮬레이션 파일 로드
style_simulation_path = "C:/Users/wnghk/Desktop/ACADEMY/2025-1.5/계량경제/스타일지수 시뮬레이션.xlsx"

try:
    # 엑셀 파일의 모든 시트 확인
    style_file = pd.ExcelFile(style_simulation_path)
    sheet_names = style_file.sheet_names
    print(f"엑셀 파일 시트 목록: {sheet_names}")
    
    # 각 시트별로 데이터 확인
    style_data = {}
    for sheet in sheet_names:
        df_sheet = pd.read_excel(style_simulation_path, sheet_name=sheet)
        style_data[sheet] = df_sheet
        
        print(f"\n--- 시트: {sheet} ---")
        print(f"데이터 크기: {df_sheet.shape}")
        print(f"컬럼명: {list(df_sheet.columns)}")
        print("첫 5행 데이터:")
        print(df_sheet.head())
        
        # 날짜 컬럼 확인
        date_columns = [col for col in df_sheet.columns if 'date' in col.lower() or '날짜' in col or col in ['Date', 'DATE']]
        if date_columns:
            print(f"날짜 컬럼: {date_columns}")
        
        # 숫자형 컬럼 확인
        numeric_columns = df_sheet.select_dtypes(include=[np.number]).columns.tolist()
        if numeric_columns:
            print(f"숫자형 컬럼: {numeric_columns}")
        
        print("-" * 50)
    
    # 주요 시트 데이터 상세 분석
    if sheet_names:
        main_sheet = sheet_names[0]  # 첫 번째 시트를 메인으로 사용
        main_data = style_data[main_sheet]
        
        print(f"\n=== 메인 데이터 ({main_sheet}) 상세 분석 ===")
        
        # 데이터 타입 확인
        print("\n데이터 타입:")
        print(main_data.dtypes)
        
        # 결측치 확인
        print(f"\n결측치 확인:")
        print(main_data.isnull().sum())
        
        # 기초 통계
        if len(main_data.select_dtypes(include=[np.number]).columns) > 0:
            print(f"\n기초 통계:")
            print(main_data.describe())
        
        # 첫 번째 컬럼이 날짜인지 확인
        first_col = main_data.columns[0]
        print(f"\n첫 번째 컬럼 '{first_col}' 샘플:")
        print(main_data[first_col].head(10))
        
        # 스타일지수 관련 컬럼 추정
        style_keywords = ['growth', 'value', 'momentum', 'quality', 'size', 'volatility', 
                         'dividend', 'small', 'large', 'mid', 'cap', '성장', '가치', '모멘텀', 
                         '품질', '배당', '변동성', '대형', '중형', '소형']
        
        style_columns = []
        for col in main_data.columns:
            for keyword in style_keywords:
                if keyword.lower() in col.lower():
                    style_columns.append(col)
                    break
        
        if style_columns:
            print(f"\n스타일지수 관련 컬럼 (추정): {style_columns}")
        
        # RSI 4계절 분류와 연결 가능성 확인
        if 'Economic_Season' in df.columns and len(main_data) > 0:
            print(f"\n=== RSI 4계절 분류와 연결 분석 ===")
            
            # 날짜 기준으로 매칭 가능한지 확인
            if first_col and main_data[first_col].dtype != 'object':
                try:
                    # 날짜 변환 시도
                    main_data['date_converted'] = pd.to_datetime(main_data[first_col])
                    print("날짜 변환 성공")
                    
                    # 기간 비교
                    style_date_range = f"{main_data['date_converted'].min()} ~ {main_data['date_converted'].max()}"
                    hmm_date_range = f"{df['date'].min()} ~ {df['date'].max()}"
                    
                    print(f"스타일지수 기간: {style_date_range}")
                    print(f"HMM 데이터 기간: {hmm_date_range}")
                    
                    # 겹치는 기간 확인
                    overlap_start = max(main_data['date_converted'].min(), df['date'].min())
                    overlap_end = min(main_data['date_converted'].max(), df['date'].max())
                    
                    if overlap_start <= overlap_end:
                        print(f"겹치는 기간: {overlap_start} ~ {overlap_end}")
                        overlap_months = (overlap_end - overlap_start).days / 30.4
                        print(f"겹치는 개월 수: 약 {overlap_months:.0f}개월")
                    else:
                        print("겹치는 기간 없음")
                        
                except Exception as e:
                    print(f"날짜 변환 실패: {e}")
        
except FileNotFoundError:
    print("❌ 파일을 찾을 수 없습니다.")
    print("파일 경로를 확인해주세요.")
    style_data = {}
    
except Exception as e:
    print(f"❌ 파일 읽기 오류: {e}")
    style_data = {}

# 스타일지수 시뮬레이션 활용 방안 제안
if style_data:
    print(f"\n=== 스타일지수 시뮬레이션 활용 방안 ===")
    print(f"🎯 RSI 4계절 분류와 결합 아이디어:")
    print(f"   • 각 계절별 최적 스타일지수 성과 분석")
    print(f"   • 계절 전환 시점의 스타일 로테이션 전략")
    print(f"   • RSI 신호와 스타일지수 수익률 상관관계")
    print(f"   • 백테스팅 및 포트폴리오 최적화")
    
    print(f"\n📊 분석 가능한 내용:")
    print(f"   1. 계절별 스타일지수 성과 순위")
    print(f"   2. 각 계절에서 아웃퍼폼하는 스타일")
    print(f"   3. 계절 전환 타이밍 최적화") 
    print(f"   4. 리스크 조정 수익률 비교")
    print(f"   5. 실제 투자 시뮬레이션")
    
    print(f"\n💡 다음 단계 제안:")
    print(f"   • 스타일지수 데이터 전처리")
    print(f"   • RSI 계절 분류와 매칭")
    print(f"   • 계절별 성과 분석")
    print(f"   • 로테이션 전략 백테스팅")
else:
    print(f"\n파일을 읽을 수 없어서 일반적인 활용 방안을 제안합니다:")
    print(f"🎯 스타일지수 + RSI 4계절 결합 전략:")
    print(f"   • 봄: 성장주, 소형주 (회복 초기)")
    print(f"   • 여름: 대형주, 고배당주 (안정 성장)")
    print(f"   • 가을: 가치주, 방어주 (하락 대비)")
    print(f"   • 겨울: 품질주, 저변동성 (안전 자산)")

print(f"\n" + "="*60)

In [ ]:
## RSI 기반 경기국면 분류 vs HMM 비교

print("=== S&P500 월별 RSI 기반 경기국면 분류 ===")

# 1. S&P500 월별 RSI 계산
def calculate_rsi(prices, window=14):
    """RSI 계산 함수"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# S&P500 가격 데이터 준비
sp500_prices = df['S&P500'].copy()
sp500_rsi_14 = calculate_rsi(sp500_prices, window=14)  # 14개월 RSI
sp500_rsi_6 = calculate_rsi(sp500_prices, window=6)   # 6개월 RSI (더 민감)

# 데이터에 RSI 추가
df['RSI_14'] = sp500_rsi_14
df['RSI_6'] = sp500_rsi_6

print(f"RSI 계산 완료:")
print(f"RSI_14 범위: {sp500_rsi_14.min():.1f} ~ {sp500_rsi_14.max():.1f}")
print(f"RSI_6 범위: {sp500_rsi_6.min():.1f} ~ {sp500_rsi_6.max():.1f}")

# 2. RSI 기반 경기국면 분류 (4가지 방법)
print(f"\n=== RSI 기반 국면 분류 방법론 ===")

# 방법 1: 전통적 RSI 임계값 (30-70)
def classify_rsi_traditional(rsi):
    """전통적 RSI 분류"""
    if rsi >= 70:
        return "과매수(버블)"
    elif rsi >= 50:
        return "상승(호황)"
    elif rsi >= 30:
        return "하락(조정)"
    else:
        return "과매도(침체)"

# 방법 2: 분위수 기반 분류
rsi_quantiles = sp500_rsi_14.quantile([0.25, 0.5, 0.75])
print(f"RSI 분위수: 25%={rsi_quantiles[0.25]:.1f}, 50%={rsi_quantiles[0.5]:.1f}, 75%={rsi_quantiles[0.75]:.1f}")

def classify_rsi_quantile(rsi, quantiles):
    """분위수 기반 RSI 분류"""
    if rsi >= quantiles[0.75]:
        return "강세장(상위25%)"
    elif rsi >= quantiles[0.5]:
        return "상승장(중상위)"
    elif rsi >= quantiles[0.25]:
        return "하락장(중하위)"
    else:
        return "약세장(하위25%)"

# 방법 3: 적응적 임계값 (이동평균 기반)
rsi_ma = sp500_rsi_14.rolling(window=24).mean()  # 2년 이동평균
rsi_std = sp500_rsi_14.rolling(window=24).std()  # 2년 표준편차

def classify_rsi_adaptive(rsi, ma, std):
    """적응적 RSI 분류"""
    if pd.isna(ma) or pd.isna(std):
        return "데이터부족"
    
    if rsi >= ma + std:
        return "극강세"
    elif rsi >= ma + 0.5*std:
        return "강세"
    elif rsi >= ma - 0.5*std:
        return "중립"
    elif rsi >= ma - std:
        return "약세"
    else:
        return "극약세"

# 방법 4: RSI 변화율 기반
rsi_change = sp500_rsi_14.diff()

def classify_rsi_momentum(rsi, rsi_change):
    """RSI 모멘텀 기반 분류"""
    if rsi >= 60 and rsi_change > 0:
        return "가속상승"
    elif rsi >= 50:
        return "상승추세"
    elif rsi >= 40:
        return "횡보"
    elif rsi_change < -5:
        return "급락"
    else:
        return "하락추세"

# 각 방법으로 분류 실행
df['RSI_traditional'] = df['RSI_14'].apply(classify_rsi_traditional)
df['RSI_quantile'] = df['RSI_14'].apply(lambda x: classify_rsi_quantile(x, rsi_quantiles))
df['RSI_adaptive'] = df.apply(lambda row: classify_rsi_adaptive(row['RSI_14'], 
                                                               rsi_ma.loc[row.name] if row.name in rsi_ma.index else None,
                                                               rsi_std.loc[row.name] if row.name in rsi_std.index else None), axis=1)
df['RSI_momentum'] = df.apply(lambda row: classify_rsi_momentum(row['RSI_14'], 
                                                               rsi_change.loc[row.name] if row.name in rsi_change.index else 0), axis=1)

# 각 방법별 분포 확인
methods = ['RSI_traditional', 'RSI_quantile', 'RSI_adaptive', 'RSI_momentum']
print(f"\n=== 각 방법별 국면 분포 ===")

for method in methods:
    print(f"\n{method}:")
    distribution = df[method].value_counts()
    for category, count in distribution.items():
        percentage = count / len(df) * 100
        print(f"  {category}: {count}개월 ({percentage:.1f}%)")

# 3. RSI vs HMM 비교 분석
print(f"\n" + "="*60)
print(f"=== RSI 분류 vs HMM 분류 비교 ===")
print(f"="*60)

# HMM 3상태 결과와 비교 (있는 경우)
if 'state_improved' in df2.columns:
    # 인덱스 맞추기
    hmm_states = df2['state_improved'].values
    rsi_traditional = df['RSI_traditional'].values
    
    # 동일한 길이로 맞추기
    min_length = min(len(hmm_states), len(rsi_traditional))
    hmm_states = hmm_states[:min_length]
    rsi_traditional = rsi_traditional[:min_length]
    
    # 교차표 생성
    comparison_df = pd.DataFrame({
        'HMM_State': hmm_states,
        'RSI_Traditional': rsi_traditional[:min_length]
    })
    
    cross_tab = pd.crosstab(comparison_df['HMM_State'], comparison_df['RSI_Traditional'], 
                           normalize='index') * 100
    
    print("HMM vs RSI 전통적 분류 교차표 (행 기준 %):")
    print(cross_tab.round(1))
    
    # 일치도 분석
    print(f"\n일치도 분석:")
    # HMM State 0 (위기) vs RSI 과매도
    # HMM State 1 (정상) vs RSI 상승/하락
    # HMM State 2 (호황) vs RSI 과매수
    
    state_mappings = {
        0: "과매도(침체)",
        1: ["하락(조정)", "상승(호황)"],
        2: "과매수(버블)"
    }
    
    matches = 0
    total = 0
    
    for i in range(min_length):
        hmm_state = hmm_states[i]
        rsi_class = rsi_traditional[i]
        total += 1
        
        if hmm_state == 0 and rsi_class == "과매도(침체)":
            matches += 1
        elif hmm_state == 1 and rsi_class in ["하락(조정)", "상승(호황)"]:
            matches += 1
        elif hmm_state == 2 and rsi_class == "과매수(버블)":
            matches += 1
    
    match_rate = matches / total * 100
    print(f"대략적 일치율: {match_rate:.1f}%")

# 4. RSI 기반 시각화
print(f"\n=== RSI 기반 시각화 ===")

# 시각화용 데이터 준비
dates_plot = df['date'].values
sp500_plot = df['S&P500'].values
rsi_plot = df['RSI_14'].values

# RSI 색상 매핑
rsi_colors = {
    "과매수(버블)": "red",
    "상승(호황)": "green", 
    "하락(조정)": "orange",
    "과매도(침체)": "blue"
}

plt.figure(figsize=(20, 12))

# 상단: S&P500 + RSI 분류
plt.subplot(3, 1, 1)
plt.plot(dates_plot, sp500_plot, color='gray', alpha=0.7, linewidth=1, label='S&P500')

for category, color in rsi_colors.items():
    mask = df['RSI_traditional'] == category
    if mask.any():
        plt.scatter(dates_plot[mask], sp500_plot[mask], 
                   c=color, label=category, alpha=0.8, s=15)

plt.title('RSI 기반 경기국면 분류 (전통적 방법)')
plt.ylabel('S&P500 Index')
plt.legend()
plt.xticks(rotation=45)

# 중간: RSI 시계열
plt.subplot(3, 1, 2)
plt.plot(dates_plot, rsi_plot, color='blue', linewidth=1)
plt.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='과매수 (70)')
plt.axhline(y=50, color='gray', linestyle='-', alpha=0.7, label='중립 (50)')
plt.axhline(y=30, color='blue', linestyle='--', alpha=0.7, label='과매도 (30)')
plt.fill_between(dates_plot, 70, 100, alpha=0.2, color='red', label='과매수 구간')
plt.fill_between(dates_plot, 0, 30, alpha=0.2, color='blue', label='과매도 구간')
plt.title('S&P500 월별 RSI (14개월)')
plt.ylabel('RSI')
plt.legend()
plt.xticks(rotation=45)

# 하단: HMM vs RSI 비교 (있는 경우)
plt.subplot(3, 1, 3)
if 'state_improved' in df2.columns:
    # HMM 상태 플롯
    hmm_dates = df2['date'].values
    hmm_sp500 = df2['S&P500'].values
    hmm_state_colors = ['blue', 'orange', 'green']
    
    for state in range(3):
        mask = df2['state_improved'] == state
        if mask.any():
            plt.scatter(hmm_dates[mask], hmm_sp500[mask], 
                       c=hmm_state_colors[state], label=f'HMM State {state}', alpha=0.8, s=15)
    
    plt.title('HMM 3상태 모델 결과 (비교용)')
    plt.ylabel('S&P500 Index')
    plt.legend()
else:
    plt.text(0.5, 0.5, 'HMM 결과 없음', ha='center', va='center', transform=plt.gca().transAxes)

plt.xlabel('Date')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\n=== RSI 기반 국면 분류의 장단점 ===")
print(f"✅ 장점:")
print(f"   • 계산이 간단하고 직관적")
print(f"   • 실시간 적용 가능")
print(f"   • 널리 알려진 기술적 지표")
print(f"   • 과매수/과매도 구간 명확")

print(f"\n❌ 단점:")
print(f"   • 단일 지표에만 의존")
print(f"   • 거시경제 요인 미반영")
print(f"   • 임계값이 자의적")
print(f"   • 가격 정보만 활용")

print(f"\n🔄 HMM 대비 특징:")
print(f"   • HMM: 다변수 + 확률적 접근")
print(f"   • RSI: 단변수 + 규칙 기반")
print(f"   • 상호 보완적 활용 가능")